# Aneruth Mohanasundaram GOR III Algorithm - 0573520
## Masters in Applied Computer Science 

In [2]:
import math
import numpy as np

In [46]:
stride = np.genfromtxt('/Users/aneruthmohanasundaram/Documents/GitHub/Advance-Bioinformatics/Assignment 1/Datasets/stride_info.txt', delimiter="\t", dtype=str)
dssp = np.genfromtxt('/Users/aneruthmohanasundaram/Documents/GitHub/Advance-Bioinformatics/Assignment 1/Datasets/dssp_info.txt', delimiter="\t", dtype=str)
cath = np.genfromtxt('/Users/aneruthmohanasundaram/Documents/GitHub/Advance-Bioinformatics/Assignment 1/Datasets/cath_info.txt', delimiter="\t", dtype=str)

#==================== GLOBAL VARIABLES
stru_dict = {'H': 0, 'E': 1, 'C': 2}
stru_list = ['H', 'E', 'C']

amino_dict = {'ALA': 0, 'ARG': 1, 'ASN': 2, 'ASP': 3, 'CYS': 4, 'GLU': 5, 'GLN': 6, 'GLY': 7, 'HIS': 8,
           'ILE': 9, 'LEU': 10, 'LYS': 11, 'MET': 12, 'PHE': 13, 'PRO': 14, 'SER': 15, 'THR': 16,
           'TRP': 17, 'TYR': 18, 'VAL': 19}

amino_list = np.array(['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLU', 'GLN', 'GLY', 'HIS', 'ILE', 'LEU','LYS',
              'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'])

amino_total_count = {}

helix_count,beta_count,coil_count = 0,0,0

## This gives us the total count of Amino Acids present in our dataset and also changes the name of our residue name to single character value

In [47]:
def count(txt_file):
    global helix_count, beta_count, coil_count, amino_total_count
    #new_list = np.empty(shape=(1,5), dtype=str )
    new_list = []
    amino_total_count = {'ALA': 0, 'ARG': 0, 'ASN': 0, 'ASP': 0, 'CYS': 0, 'GLU': 0, 'GLN': 0, 'GLY': 0, 'HIS': 0,
           'ILE': 0, 'LEU': 0, 'LYS': 0, 'MET': 0, 'PHE': 0, 'PRO': 0, 'SER': 0, 'THR': 0,
           'TRP': 0, 'TYR': 0, 'VAL': 0}
    
    helix_count,beta_count,coil_count = 0,0,0
    txt_file = np.char.replace(txt_file, 'Helix', 'H')
    txt_file = np.char.replace(txt_file, 'Beta', 'E')
    txt_file = np.char.replace(txt_file, 'Other', 'C')
    txt_file = np.char.replace(txt_file, 'Coil', 'C')

    for i in range(len(txt_file)):  
        if txt_file[i][3] in list(amino_dict.keys()):
            c = amino_total_count.get(txt_file[i][3])
            c += 1
            amino_total_count.update({txt_file[i][3]: c})
            if txt_file[i][4] == 'H':helix_count += 1
            elif txt_file[i][4] == 'E':beta_count += 1
            elif txt_file[i][4] == 'C':coil_count += 1
            new_list.append(txt_file[i])
    return new_list

In [48]:
def separate_protein_class(dataset):
   ''' Return as list where we pass our dataset given '''
    dataset = np.array(dataset, dtype=str)
    _, indices = np.unique(dataset[:, 0], return_index=True)
    indices = np.sort(indices)
    new_list = np.split(dataset, indices)[1:]
    return new_list

# Functions For Self information Table 

In [51]:
def SelfInfoTable():
    return np.zeros(shape=(20,3),dtype=int).tolist()  # 20 proteins are added in the list
    
def SelfCountTable(table, residue, structure, count):
    table[residue][structure] = count

def SelfCountAssignTable(table, residue, structure):
    return table[residue][structure]

# Functions For Residual Information Table

In [52]:
def residual_create_table():
    # This creates a list for 20 amino acids present where we later append the values of our MCC and Q3 score    
    return np.zeros(shape=(20,18,20,3),dtype=int).tolist() 

def residual_setter(table, jres, m, mres, jstruct, count):
    table[jres][m][mres][jstruct] = count

def residual_getter(table, jres, m, mres, jstruct):
    return table[jres][m][mres][jstruct]

In [53]:
def generate(aGroup):
    self_table = SelfInfoTable()
    residual_table = residual_create_table()
    ''' Gets the values generated from self info table and residual table '''
    for protein_list in aGroup:
        for j in range(len(protein_list)):
            self_name = protein_list[j][-2]  
            self_stru = protein_list[j][-1]  
            # update the self information table
            count = SelfCountAssignTable(self_table, amino_dict[self_name], stru_dict[self_stru])
            SelfCountTable(self_table, amino_dict[self_name], stru_dict[self_stru], count + 1)
            # update the pair information table
            sequence0 = protein_list[j][-3]        # residue sequence code at the location of j
            for m in np.arange(-8,9,1).tolist():
                if m == 0:continue  # except for base residue
                if j + m < 0 or j + m > len(protein_list) - 1:continue                    
                sequence1 = (protein_list[j + m])[-3]       
                m0 = int(sequence1) - int(sequence0)        # relative location of sequence code
                # if our sequence not in range between -8 to 8 excluding 0 then it'll tends to break so to avoid we are continuing the loop.
                if m0 not in np.arange(-8,9,1).tolist():continue 
                residual_name = (protein_list[j + m])[-2]
                # update the pair information table
                count = residual_getter(residual_table, amino_dict[self_name], m0 + 8, amino_dict[residual_name], stru_dict[self_stru])
                residual_setter(residual_table, amino_dict[self_name], m0 + 8, amino_dict[residual_name], stru_dict[self_stru], count + 1)
        # create_table(protein, self_table, residual_table)
    return self_table, residual_table

In [54]:
def compute_self_information(self_table, residue_name):
    ''' This function is used to store the value of specific Structure type.
    It counts each values of H,E and C for our amino acids present.
    For example: If we have a acid ILE then for it the H,E and C count will appends to the list.''' 
    I = []      
    for stru in stru_list:      
        fsr = SelfCountAssignTable(self_table, amino_dict[residue_name], stru_dict[stru])     
        fnsr,fns,fs = 0,0,0       
        for s in stru_list:
            if s != stru: # If our value is not present in our structure list then we assign the fsnr and increment it with our self count table
                fnsr = fnsr + SelfCountAssignTable(self_table, amino_dict[residue_name], stru_dict[s])
        for i in list(amino_dict.keys()): # Similar to our structure type we are checking our amino acids and incrementing
            for s in stru_list:
                if s != stru:
                    fns = fns + SelfCountAssignTable(self_table, amino_dict[i], stru_dict[s])
                else:
                    fs = fs + SelfCountAssignTable(self_table, amino_dict[i], stru_dict[s])
        if fnsr == 0: # If our values hits zero we assign it as infinity. 
            i = math.inf
        else:
            i = math.log(fsr/fnsr) + math.log(fns/fs)
        I.append(i)
    return I

In [55]:
def predict(protein, j, self_table, residual_table):
    ''' Centered on the formula: This procedure computes the self knowledge of a given residue: I(ΔSj ;Rj ).
    This formula is used to calculate the propensity of our self information table for our residue name. '''
    self_name = (protein[j])[-2]
    
    ''' # Computing self-information propensity from the given formula I(ΔSj ;Rj ) which involves calculating the pair frequencies of residues from our self info table.'''
    I = compute_self_information(self_table, self_name) 
    sequence0 = int(protein[j][2])
    for a in range(3):
        stru = stru_list[a]
        for m in np.arange(-8,9,1).tolist():
            # avoid index out of range
            if m == 0 :continue
            if j + m < 0 or j + m > len(protein) - 1:continue
            sequence1 = int((protein[j + m])[2])
            m0 = sequence1 - sequence0  # the minus of two residues sequence code
            if m0 not in np.arange(-8,9).tolist():continue
            self_name = (protein[j])[-2]
            residual_name = (protein[j + m])[-2]
            residue0 = residual_getter(residual_table, amino_dict[self_name], m0 + 8, amino_dict[residual_name], stru_dict[stru])
            residue1,residue2 = 0,0
            residue3 = SelfCountAssignTable(self_table, amino_dict[self_name], stru_dict[stru])
            for s in stru_list:
                if s != stru:
                    residue1 = residue1 + residual_getter(residual_table, amino_dict[self_name], m0 + 8, amino_dict[residual_name], stru_dict[s])
                    residue2 = residue2 + SelfCountAssignTable(self_table, amino_dict[residual_name], stru_dict[s])
            if residue1 == 0:
                I[a] = math.inf
                break
            try:
                I[a] = I[a] + math.log(residue0 / residue1) + math.log(residue2 / residue3)
            except ValueError:
                I[a] = -math.inf
                break
    # considering the biggest values
    max_value = max(I)
    for i in range(3):
        if I[i] == max_value:
            i = i
            break
    return stru_list[i]

In [56]:
def predict_dataset(nested_list, result_list, data_self_table, data_residual_table):
    for i in range(len(nested_list)):
        self_table = SelfInfoTable()
        residual_table = residual_create_table()
        for j in range(len(nested_list)):
            if i == j:
                protein_list = nested_list[i]
                for j in range(len(protein_list)):
                    self_name = protein_list[j][-2]  # the residue name of j
                    self_stru = protein_list[j][-1]  # the secondary structure of j
                    # update the self information table
                    count = SelfCountAssignTable(self_table, amino_dict[self_name], stru_dict[self_stru])
                    SelfCountTable(self_table, amino_dict[self_name], stru_dict[self_stru], count + 1)
                    # update the pair information table
                    sequence0 = protein_list[j][-3]        # the residue sequence code at the location of j
                    for m in range(-8, 9, 1):
                        if m == 0:  # if we get a base pair of residue then we continue looping
                            continue
                        if j + m < 0 or j + m > len(protein_list) - 1:
                            continue
                        sequence1 = (protein_list[j + m])[-3]
                        m0 = int(sequence1) - int(sequence0)        # assigning relative location of sequence code
                        if m0 not in np.arange(-8,9,1).tolist():continue    # avoid sequence break
                        residual_name = (protein_list[j + m])[-2]
                        # update the pair information table
                        count = residual_getter(residual_table, amino_dict[self_name], m0 + 8, amino_dict[residual_name], stru_dict[self_stru])
                        residual_setter(residual_table, amino_dict[self_name], m0 + 8, amino_dict[residual_name], stru_dict[self_stru], count + 1)
                break
        pred = []
        for j in range(len(protein_list)):
            pred.append(predict(protein_list, j, data_self_table, data_residual_table))
        result_list.append(pred)

In [57]:
def score(predict_value, facts):
    output = []
    # traverse both the prediction and actual structure and comparing them
    for i, j in zip(predict_value, facts): # We use zip to join both the list and fetching the respective values required 
        correct = 0
        mcc_val = np.zeros(shape=(3,4),dtype=int).tolist() # Creating a table where we are appending the values of our MCC Coef.
        for p, f in zip(i, j): 
            if p == f[-1]:
                correct += 1
            for k in range(3):
                residue = stru_list[k]
                if f[-1] == residue:
                    if f[-1] == p:  # True Positive
                        mcc_val[k][0] += 1
                    else:   # False Negative
                        mcc_val[k][2] += 1
                else:
                    if p == residue:    # False Positive
                        mcc_val[k][1] += 1
                    else:
                        mcc_val[k][3] += 1  # True Negative
        MCC = []
        for table in mcc_val:
            TP = table[0]
            FP = table[1]
            FN = table[2]
            TN = table[3]
            try:
                mcc = (TP*TN - FP*FN)/math.sqrt((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))
            except ZeroDivisionError:
                mcc = (TP*TN - FP*FN)
            MCC.append(mcc)
        Q3 = correct/len(i)
        output.append([Q3, MCC])
    return output

# predict the protein family it is associated with the MCC.Helix ,MCC.Sheet and MCC.Coil

In [58]:
def pFamily(scores):
    def criterion(score):
        # Alpha calcualtion
        if abs(score[1]) <= 0.03:
            return 'Alpha'
        # Beta calculation
        elif abs(score[0]) <= 0.03:
            return 'Beta'
        # Mixed residue calculation
        else:
            return 'Alpha/beta'
    family = []
    for score in scores:
        family.append(criterion(score[-1]))
    return family

In [59]:
def accuracy(pFamily, facts):
    count = 0
    for i, j in zip(pFamily, facts):
        if i == j[-1]:count += 1
    return count/len(facts)

 # A method to implement GOR algorithm first and calculate our Q3 and MCC score

In [60]:
def normalGorOutput(stride,dssp,cath):
    stride_list = np.char.strip(stride)
    dssp_list = np.char.strip(dssp)
    cath_list = np.char.strip(cath)

    print('\nThe division of structure type in the STRIDE dataset is as follows: ')
    stride_list = count(stride_list)
    print('The Helix count is: ' + str(helix_count),'\nThe Beta count is:' + str(beta_count),'\nThe Coil count is:' + str(coil_count),'\n',amino_total_count)
    
    print('\nThe division of structure type in the DSSP dataset is as follows: ')
    dssp_list = count(dssp_list)
    print('The Helix count is: ' + str(helix_count),'\nThe Beta count is:' + str(beta_count),'\nThe Coil count is:' + str(coil_count),'\n',amino_total_count)

    stride_nest = separate_protein_class(stride_list)
    dssp_nest = separate_protein_class(dssp_list)   

    self_table_stride, residual_table_stride = generate(stride_nest)
    self_table_dssp, residual_table_dssp = generate(dssp_nest)

    pred_stride, pred_dssp = [], []
    stride_result, dssp_result = [], [[]]
    
    predict_dataset(stride_nest, pred_stride, self_table_stride, residual_table_stride)
    predict_dataset(dssp_nest, pred_dssp, self_table_dssp, residual_table_dssp)

    scores_stride = score(pred_stride, stride_nest)
    scores_dssp = score(pred_dssp, dssp_nest)

    val1,val2 = 0,0

    for el1, el2 in zip(scores_stride, scores_dssp):
        if el1[0] >= 0.5:val1 += 1
        if el2[0] >= 0.5:val2 += 1

    print('\nThe overall Q3 for stride without leaving out protein in the dataset is', 100*val1/len(scores_stride), '%.')
    print('The overall Q3 for dssp without leaving out protein in the dataset is', 100*val2/len(scores_dssp), '%.')

    predf_stride = pFamily(scores_stride)
    predf_dssp = pFamily(scores_dssp)

    print(f"\nThe accuracy of prediction for protein family in stride dataset without leaveone out method is {(accuracy(predf_stride, cath_list)*100)}")
    print(f"The accuracy of prediction for protein family in dssp dataset without leaveone out method is {(accuracy(predf_dssp, cath_list)*100)}")# predict the protein family

# Leave One Out Method 

## This method is calculated by where we eleminate a residue for example we have a residue name Helix where it has a certain count and we are removing it from the main dataset and we are using the excluded dataset as our main dataset and performing normal GOR III method.

In [44]:
def leaveOneOut(dataset):
    '''
    dataset --> takes a list value
    Since our dataset takes as a list type and it's enumerated with our index value and pops the values based on the index position present in our datase.
    '''
    for idx,row in enumerate(dataset): 
        if (row[3] == 'ILE' and row[4] == 'Helix'):
            dataset.pop(idx)
        elif (row[3] == 'ILE' and row[4] == 'Beta'):
            dataset.pop(idx) 
        elif (row[3] == 'ILE' and row[4] == 'Coil') or (row[3] == 'ILE' and row[4] == 'Other'):
            dataset.pop(idx)
    return dataset

# Calculating all Q3 and MCC score after Leave one out method 

In [61]:
def LeaveOneOutImplements(stride,dssp,cath):
    stride_leave = np.char.strip(stride)
    dssp_leave = np.char.strip(dssp)
    cath_list = np.char.strip(cath)
    
    print('\nThe division of structure type in the STRIDE dataset is as follows: ')
    sl = leaveOneOut(stride_leave.tolist())
    stride_list = count(sl)
    print('The Helix count is: ' + str(helix_count),'\nThe Beta count is:' + str(beta_count),'\nThe Coil count is:' + str(coil_count),'\n',amino_total_count)
    
    print('\nThe division of structure type in the DSSP dataset is as follows: ')
    dl = leaveOneOut(dssp_leave.tolist())
    dssp_list = count(dl)
    print('The Helix count is: ' + str(helix_count),'\nThe Beta count is:' + str(beta_count),'\nThe Coil count is:' + str(coil_count),'\n',amino_total_count)

    stride_leave_nest = separate_protein_class(stride_list)
    dssp_leave_nest = separate_protein_class(dssp_list)   

    self_table_stride, residual_table_stride = generate(stride_leave_nest)
    self_table_dssp, residual_table_dssp = generate(dssp_leave_nest)

    pred_stride, pred_dssp = [], []
    stride_result, dssp_result = [], [[]]

    predict_dataset(stride_leave_nest, pred_stride, self_table_stride, residual_table_stride)
    predict_dataset(dssp_leave_nest, pred_dssp, self_table_dssp, residual_table_dssp)

    scores_stride = score(pred_stride, stride_leave_nest)
    scores_dssp = score(pred_dssp, dssp_leave_nest)

    count1 = 0
    count2 = 0

    for el1, el2 in zip(scores_stride, scores_dssp):
        if el1[0] >= 0.5:
            count1 += 1
        if el2[0] >= 0.5:
            count2 += 1

    print('\nThe overall Q3 for stride with leaving out protein in the dataset is', 100*count1/len(scores_stride), '%.')
    print('The overall Q3 for dssp with leaving out protein in the dataset is', 100*count2/len(scores_dssp), '%.')

    predf_stride = pFamily(scores_stride)
    predf_dssp = pFamily(scores_dssp)

    print(f"\nThe accuracy of prediction for protein family in stride dataset with leaveone out method is {(accuracy(predf_stride, cath_list)*100)}")
    print(f"The accuracy of prediction for protein family in dssp dataset with leaveone out method is {(accuracy(predf_dssp, cath_list)*100)}")# predict the protein family

# Main function to print all values where we call it in one function

In [62]:
if __name__ == "__main__":
    normalGorOutput(stride,dssp,cath)
    print('\nQ3 and Accuracy after Leave one out method')
    LeaveOneOutImplements(stride,dssp,cath)


The division of structure type in the STRIDE dataset is as follows: 
The Helix count is: 25755 
The Beta count is:16148 
The Coil count is:29170 
 {'ALA': 5963, 'ARG': 3665, 'ASN': 3053, 'ASP': 4244, 'CYS': 1103, 'GLU': 4816, 'GLN': 2849, 'GLY': 5119, 'HIS': 1715, 'ILE': 4034, 'LEU': 6449, 'LYS': 4213, 'MET': 1260, 'PHE': 2890, 'PRO': 3181, 'SER': 4148, 'THR': 3937, 'TRP': 1007, 'TYR': 2467, 'VAL': 4960}

The division of structure type in the DSSP dataset is as follows: 
The Helix count is: 24862 
The Beta count is:15694 
The Coil count is:30031 
 {'ALA': 5961, 'ARG': 3665, 'ASN': 3053, 'ASP': 4242, 'CYS': 626, 'GLU': 4815, 'GLN': 2849, 'GLY': 5116, 'HIS': 1716, 'ILE': 4034, 'LEU': 6449, 'LYS': 4212, 'MET': 1261, 'PHE': 2890, 'PRO': 3181, 'SER': 4146, 'THR': 3937, 'TRP': 1007, 'TYR': 2467, 'VAL': 4960}

The overall Q3 for stride without leaving out protein in the dataset is 62.550607287449395 %.
The overall Q3 for dssp without leaving out protein in the dataset is 66.39676113360323 %.

### The results show that there is a substantial increase in the accuracy of the stride dataset, 62 percent vs 66 percent when one is removed, but there is just a small increase in the accuracy of the DSSP dataset. This suggests an error in my algorithm, but after attempting multiple methods, this inconsistency persists.